In [4]:
import numpy as np

In [46]:
#test array/graph from https://people.sc.fsu.edu/~jburkardt/m_src/rcm/rcm.html
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
print(graph)

[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]


In [43]:
#to generate rand arr
N = 10
b = np.random.randint(2,size=(N,N))
#b_symm = np.array(np.ceil((b + b.T)/2), dtype=int)
#np.fill_diagonal(b_symm, 0)
#b_symm
print(b+b.T)

[[2 0 1 2 0 1 1 0 2 2]
 [0 2 2 1 1 2 0 0 2 0]
 [1 2 2 0 1 2 2 2 1 1]
 [2 1 0 2 0 0 2 1 2 1]
 [0 1 1 0 0 0 0 2 1 1]
 [1 2 2 0 0 0 1 0 2 1]
 [1 0 2 2 0 1 0 2 1 1]
 [0 0 2 1 2 0 2 0 1 0]
 [2 2 1 2 1 2 1 1 0 1]
 [2 0 1 1 1 1 1 0 1 2]]


In [91]:
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.csgraph import reverse_cuthill_mckee

In [92]:
c_g = csr_matrix(graph)
print(c_g)
perm = reverse_cuthill_mckee(c_g, True)
perm

  (0, 2)	1
  (0, 7)	1
  (0, 8)	1
  (1, 2)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (2, 0)	1
  (2, 1)	1
  (3, 7)	1
  (3, 8)	1
  (4, 6)	1
  (4, 7)	1
  (5, 1)	1
  (5, 6)	1
  (6, 1)	1
  (6, 4)	1
  (6, 5)	1
  (7, 0)	1
  (7, 1)	1
  (7, 3)	1
  (7, 4)	1
  (8, 0)	1
  (8, 3)	1


array([4, 3, 6, 5, 7, 8, 1, 0, 2], dtype=int32)

In [93]:
g = graph[perm]
print(g)
print(c_g[perm])

[[0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 1 1]
 [0 1 0 0 1 1 0 0 0]
 [0 1 0 0 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 1 1 1 0]
 [0 0 1 0 0 0 0 1 1]
 [1 1 0 0 0 0 0 0 0]]
  (0, 7)	1
  (0, 6)	1
  (1, 8)	1
  (1, 7)	1
  (2, 5)	1
  (2, 4)	1
  (2, 1)	1
  (3, 6)	1
  (3, 1)	1
  (4, 4)	1
  (4, 3)	1
  (4, 1)	1
  (4, 0)	1
  (5, 3)	1
  (5, 0)	1
  (6, 7)	1
  (6, 6)	1
  (6, 5)	1
  (6, 2)	1
  (7, 8)	1
  (7, 7)	1
  (7, 2)	1
  (8, 1)	1
  (8, 0)	1


In [94]:
graph = [
[0, 1 , 2, 0],
[0, 0, 0, 1],
[2, 0, 0, 3],
[0, 0, 0, 0]
]
graph = csr_matrix(graph)
print(graph)


  (0, 1)	1
  (0, 2)	2
  (1, 3)	1
  (2, 0)	2
  (2, 3)	3


In [95]:
perm = reverse_cuthill_mckee(graph)
print(perm)
graph[perm].toarray()

[3 2 1 0]


array([[0, 0, 0, 0],
       [2, 0, 0, 3],
       [0, 0, 0, 1],
       [0, 1, 2, 0]], dtype=int32)

In [16]:
print(graph)
graph.shape[0]
np.array([-1]*graph.shape[0])

[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]


array([-1, -1, -1, -1, -1, -1, -1, -1, -1])

In [143]:
'''graphs collection for testing'''
#test array/graph from https://people.sc.fsu.edu/~jburkardt/m_src/rcm/rcm.html
graph = np.array([
            [0, 0, 1, 0, 0, 0, 0, 1, 1],
            [0, 0, 1, 0, 0, 1, 1, 1, 0],
            [1, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 1, 1, 0, 0, 0],
            [1, 1, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 1, 0, 0, 0, 0, 0]
             ])
print(graph)

#nauru graph
graph_1 = np.zeros((24,24))
graph_1[0][1] = graph_1[0][5] = graph_1[0][21] = \
graph_1[1][3] = graph_1[1][15] = \
graph_1[2][3] = graph_1[2][4] = graph_1[2][23] = \
graph_1[3][9] = graph_1[4][5] = graph_1[4][17] = \
graph_1[5][11] = graph_1[6][7] = graph_1[6][11] = graph_1[6][19] = \
graph_1[7][9] = graph_1[7][13] = \
graph_1[8][9] = graph_1[8][10] = graph_1[8][22] = \
graph_1[10][11] = graph_1[10][16] = \
graph_1[12][13] = graph_1[12][17] = graph_1[12][18] = \
graph_1[13][15] = graph_1[14][15] = graph_1[14][16] = graph_1[14][20] = \
graph_1[16][17] = graph_1[18][19] = graph_1[18][23] = \
graph_1[19][21] = graph_1[20][21] = graph_1[20][22] = \
graph_1[22][23] = 1
graph_1 += graph_1.T
print(graph_1)

[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]
[[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0

In [112]:
'''start of eli'''
'''preliminaries:
- node == vertex ("vertices" for plural)
- nodes' index start at 0
- all graphs and trees are represented in matrix farm
- valency is the sum of the edges' weights of a node, i.e. the sum of the row in a corresponding index, in numpy:
    np.sum(graph[node_idx])
'''
#clique checker:

n = graph.shape[0] #number of nodes
e = [] #placeholder node vector for elimination ordering
w = np.array([1]*n) #weight vector for merge forest
merge_forest = np.zeros((n,n)) #merge forest for assessment criteria
modified = np.array([0]*n) #modified = 1, otherwise 0
fill_count = 0

#normalize stage
#for now, cyclic ordering assumption: start from the 1st index to last idx, hence for-loop
for i in range(n):
    #recalculate all of the values
    valencies = np.array([np.sum(graph[j]) for j in range(n)]) #needs to recalculate the valency for each update due to the graph-change
    mean_valency = np.sum(valencies)/n #get mean valency
    max_valency = np.max(valencies) #get max valency
    valency = valencies[i] #get vertex's valency
    m = np.min([mean_valency, n**(1/4) + 3])
    neighboors = np.where(graph[i] == 1)[0] #get the neighboors of i
    
    #check all of the conditions based on the valency
    if valency == n-1:
        e.append(i) #add to the end
        #graph[i] = graph[:,i] = 0  #remove from graph
        graph = np.delete(graph, i, 0) #delete from graph -- this should be the proper deletion method, though not sure if it's faster
        graph = np.delete(graph, i, 1)
    elif (valency > n/2) and (valency == max_valency):
        e.append(i)
        graph = np.delete(graph, i, 0)
        graph = np.delete(graph, i, 1)
    elif valency <= 1:
        e.insert(0, i) #place vertex first
        graph = np.delete(graph, i, 0)
        graph = np.delete(graph, i, 1)
    elif valency == 2:
        e.insert(0, i)
        
        graph[neighboors[0]][neighboors[1]] = graph[neighboors[1]][neighboors[0]] = 1 #make edge between them -- fill the value of the cell with 1
        graph = np.delete(graph, i, 0)
        graph = np.delete(graph, i, 1)
    #elif valency <= m and clique:
    #elif valency <= m and subset of j
    modified[i] = 0 #set as unmodified
    
    #set the neighboors as modified


In [144]:
print(graph)
graph = np.delete(graph, 1, 1)
graph = np.delete(graph, 1, 0)
print(graph)

[[0 0 1 0 0 0 0 1 1]
 [0 0 1 0 0 1 1 1 0]
 [1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 1 0 0]
 [0 1 0 0 1 1 0 0 0]
 [1 1 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]]
[[0 1 0 0 0 0 1 1]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1]
 [0 0 0 0 0 1 1 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 1 0 0 0]
 [1 0 1 1 0 0 0 0]
 [1 0 1 0 0 0 0 0]]
